Esse código tem como objetivo extrair informações de um artigo científico em PDF.

Ele está ajustado para extrair os dados de artigos no formato Elsevier.

In [ ]:
 # Instala as bibliotecas necessárias
!pip install PyMuPDF

import fitz
import nltk
from collections import Counter
import re

from google.colab import drive
drive.mount('/content/drive')

nltk.download('punkt')

 # Extrai o texto do PDF
def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text("text")
    return text

 # Extrai a data de publicação
def extract_date(lines):
    for line in lines:
        if 'Available online' in line:
            date_line = line
            date = date_line.split('Available online')[-1].strip()
            return date
    return ""

 # Preparação para extrair cada informação
def get_article_info(text):
    lines = text.splitlines()

    title = ""
    subtitle = ""
    author = ""
    date = ""
    abstract = ""
    references = []
    contributions = ""
    ref_texts = {}

    # Extrai o título e o subtítulo
    for i, line in enumerate(lines):
        if 'Review Article' in line:
            title_index = i + 1
            subtitle_index = i + 2
            title = lines[title_index].strip()
            subtitle = lines[subtitle_index].strip()
            break

    # Extrai os autores
    for line in lines:
        if '∗' in line:
            author = line.strip().replace('∗', '')
            break

    # Extrai data
    date = extract_date(lines)

    # Extrai o resumo
    abstract_start = False
    for line in lines:
        if 'a b s t r a c t' in line.lower():
            abstract_start = True
            continue
        if abstract_start:
            if line.strip() and not re.match(r'© 2024', line, re.IGNORECASE):
                abstract += line.strip() + " "
            else:
                break
    abstract = abstract.strip()

    # Extrai as referências contando números entre []
    ref_pattern = re.compile(r'\[(\d+)\]')
    for line in lines:
        matches = ref_pattern.findall(line)
        for match in matches:
            refs = match.split(',')
            references.extend(refs)

    ref_counts = Counter(references)

    # Extrai nomes das referências
    ref_section_start = False
    ref_section_lines = []
    for line in lines:
        if 'References' in line or 'REFERENCES' in line:
            ref_section_start = True
            continue
        if ref_section_start:
            ref_section_lines.append(line)

    # Assegura que as referências estejam no formato correto
    ref_num_pattern = re.compile(r'^\s*\[(\d+)\]\s*(.+)$')
    for line in ref_section_lines:
        match = ref_num_pattern.match(line.strip())
        if match:
            ref_num, ref_text = match.groups()
            ref_texts[ref_num] = ref_text

    # Extrai as contribuições
    contributions_start = False
    for line in lines:
        if 'Conclusion' in line:
            contributions_start = True
            continue
        if contributions_start:
            if 'Declaration of competing interest' in line:
                break
            contributions += line.strip() + " "
    contributions = contributions.strip()


    top_references = [(ref, count, ref_texts.get(ref, 'Referência não encontrada')) for ref, count in ref_counts.most_common(3)]

  # Retorna um dicionário com as informações necessárias
    return {
        "title": title,
        "subtitle": subtitle,
        "author": author,
        "date": date,
        "abstract": abstract,
        "ref_count": len(ref_counts),
        "top_references": top_references,
        "contributions": contributions
    }

  # Processa o arquivo, extrai e imprime as informações
def main(pdf_path):
    text = extract_text_from_pdf(pdf_path)
    info = get_article_info(text)

    print("Título do artigo:", info["title"])
    print("Subtítulo do artigo:", info["subtitle"])
    print("Autor do artigo:", info["author"])
    print("Data de publicação:", info["date"])
    print("Resumo:", info["abstract"])
    print("Quantidade de referências:", info["ref_count"])
    print("Referências mais mencionadas:", info["top_references"])
    for ref, count, name in info["top_references"]:
        print(f"A referência [{ref}] foi mencionada {count} vezes: {name}")
    print("Aporte científico da pesquisa:", info["contributions"])

# Executa a função com o caminho fornecido para o PDF
# Lembre-se substituir "minhapasta" pelo nome de sua pasta e "artigo1" pelo nome do seu artigo!
pdf_path = "/content/drive/My Drive/minhapasta/artigo1.pdf"
main(pdf_path)